# Assessment 1: Group 6


**Group Members:**

> Abrigo, Nathanael\
> Buhay, Kyle Andrei\
> Cruz, Kristel Lenci\
> Entrata, Joshua Kyle


In [72]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pycountry
import pycountry_convert as pc

import warnings

In [73]:
warnings.filterwarnings("ignore")

In [74]:
pd.set_option('display.max_columns', None)

## `Part 1: Data Wrangling`


### **Data Cleaning**


#### Loading Datasets


In [75]:
# Load FAO dataset
df_fao = pd.read_csv('FAO.csv', encoding='iso-8859-1')

# Load FAOSTAT dataset
df_faostat = pd.read_csv('FAOSTAT.csv')

#### Data Dictionary


`FAO Dataset`


In [76]:
df_fao.sample(3)

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
783,ARM,1,Armenia,2905,Cereals - Excluding Beer,5142,Food,1000 tonnes,40.07,45.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,566.0,566.0,494.0,508.0,530.0,541.0,534.0,520.0,498.0,483.0,454.0,469.0,506.0,460.0,476.0,445.0,447.0,416.0,401.0,429.0,393,406
14605,OMN,221,Oman,2511,Wheat and products,5521,Feed,1000 tonnes,21.51,55.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
21305,ZMB,251,Zambia,2658,"Beverages, Alcoholic",5142,Food,1000 tonnes,-13.13,27.85,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,5.0,5.0,4.0,6.0,7.0,6.0,7,0


In [77]:
# Create a list of descriptions
description_list = [
    'Abbreviation of Area',
    'Full name of the country or region',
    'ISO country code or region abbreviation',
    'Numeric code for the country or region',
    'Numeric code for the item',
    'Name of the food item or product',
    'Numeric code for the element',
    'Type of data recorded',
    'Measurement unit for the recorded data',
    'Latitude coordinate of the location',
    'Longitude coordinate of the location',
]

In [78]:
# Function to generate a data dictionary
def generate_data_dictionary(df):
    data_dict = {
        'Column Name': [],
        'Data Type': [],
        'Description': []
    }
    
    for i, column in enumerate(df.columns):
        data_dict['Column Name'].append(column)
        data_dict['Data Type'].append(df[column].dtype)
        # Provide a description based on the description list or a default value
        description = description_list[i] if i < len(description_list) else 'N/A'
        # If the column name starts with 'Y' and is a year, format accordingly
        if column.startswith('Y') and column[1:].isdigit():
            description = f'Data for the year {column[1:]}.'
        data_dict['Description'].append(description)
    
    return pd.DataFrame(data_dict)

In [79]:
# Generate data dictionary
fao_data_dict = generate_data_dictionary(df_fao)

# Display data dictionary
fao_data_dict

,Column Name,Data Type,Description
0,Area Abbreviation,object,Abbreviation of Area
1,Area Code,int64,Full name of the country or region
2,Area,object,ISO country code or region abbreviation
3,Item Code,int64,Numeric code for the country or region
4,Item,object,Numeric code for the item
...,...,...,...
58,Y2009,float64,Data for the year 2009.
59,Y2010,float64,Data for the year 2010.
60,Y2011,float64,Data for the year 2011.
61,Y2012,int64,Data for the year 2012.


`FAOSTAT Dataset`


In [80]:
# Create a list of descriptions
description_mapping=[
    "Numeric Code for Domain",
    "Population Type",
    "Numeric code for the country or region",
    "Full name of the country or region",
    "Numeric code for the element",
    "Element Type",
    "Numeric code for the item",
    "Object of focus",
    "Code for the Year",
    "Time of the Year",
    "Quantity of item",
    "Value of the item",
    "Flag",
    "Sources of item",
    "Note for item",
]

In [81]:
# Initialize a counter
j = 0

# Empty list to store the data for the DataFrame
data = []

for i in df_faostat.columns:
  data.append({'Column Name': i, 'Data Type': df_faostat[i].dtype, 'Data Description': description_mapping[j]})
  j += 1

In [82]:
# Create the DataFrame from the list of dictionaries
faostat_data_dict = pd.DataFrame(data)

# Display the data dictionary
faostat_data_dict

,Column Name,Data Type,Data Description
0,Domain Code,object,Numeric Code for Domain
1,Domain,object,Population Type
2,Area Code,int64,Numeric code for the country or region
3,Area,object,Full name of the country or region
4,Element Code,int64,Numeric code for the element
5,Element,object,Element Type
6,Item Code,int64,Numeric code for the item
7,Item,object,Object of focus
8,Year Code,int64,Code for the Year
9,Year,int64,Time of the Year


#### Handling Missing Values


`FAO Dataset`


In [83]:
# Checking row null value per column
fao_null_values = {col: df_fao[col].isnull().sum() for col in df_fao.columns if df_fao[col].isnull().sum() > 0}

fao_null_values

{'Y1961': 3539,
 'Y1962': 3539,
 'Y1963': 3539,
 'Y1964': 3539,
 'Y1965': 3539,
 'Y1966': 3539,
 'Y1967': 3539,
 'Y1968': 3539,
 'Y1969': 3539,
 'Y1970': 3539,
 'Y1971': 3539,
 'Y1972': 3539,
 'Y1973': 3539,
 'Y1974': 3539,
 'Y1975': 3539,
 'Y1976': 3539,
 'Y1977': 3539,
 'Y1978': 3539,
 'Y1979': 3539,
 'Y1980': 3539,
 'Y1981': 3539,
 'Y1982': 3539,
 'Y1983': 3539,
 'Y1984': 3539,
 'Y1985': 3539,
 'Y1986': 3539,
 'Y1987': 3539,
 'Y1988': 3539,
 'Y1989': 3539,
 'Y1990': 3415,
 'Y1991': 3415,
 'Y1992': 987,
 'Y1993': 612,
 'Y1994': 612,
 'Y1995': 612,
 'Y1996': 612,
 'Y1997': 612,
 'Y1998': 612,
 'Y1999': 612,
 'Y2000': 349,
 'Y2001': 349,
 'Y2002': 349,
 'Y2003': 349,
 'Y2004': 349,
 'Y2005': 349,
 'Y2006': 104,
 'Y2007': 104,
 'Y2008': 104,
 'Y2009': 104,
 'Y2010': 104,
 'Y2011': 104}

In [84]:
df_fao[df_fao.isnull().any(axis=1)].head(1)

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
679,ARM,1,Armenia,2511,Wheat and products,5521,Feed,1000 tonnes,40.07,45.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,21.0,24.0,25.0,10.0,32.0,58.0,55.0,78.0,20.0,20.0,42.0,69.0,59.0,46.0,67.0,57.0,56.0,61.0,65.0,92,93


In [85]:
print(f'Number of rows with null values: {len(df_fao[df_fao.isnull().any(axis=1)])}')

Number of rows with null values: 3539


`FAOSTAT Dataset`


In [86]:
faostat_null_values = {col: df_faostat[col].isnull().sum() for col in df_faostat.columns if df_faostat[col].isnull().sum() > 0}

faostat_null_values

{}

**Explanation & Justification:**

> We handled missing data in the `FAO` and `FAOSTAT` dataset specifically for the years 1961 - 1991 by retaining the null values rather than replacing them with -1. Since the "y{YEAR_NUM}" columns are the only ones with null values, it is acceptable to leave these nulls as they are. Replacing these nulls with -1 could affect the summarization of statistics and affect the accuracy of visualizations.

> Retaining null values preserves the integrity of our statistical summaries, such as mean and standard deviation calculations, which could be skewed by arbitrary replacement values like -1. By keeping nulls, we avoid misrepresenting missing data as zeroes, thus ensuring that our plots and analyses accurately reflect the data's true nature and maintain clarity in our visualizationons.

#### Removing & Checking Duplicates


`FAO Dataset`


In [87]:
# Duplicates in FAO
dupe_fao_sum= df_fao.duplicated().sum()

print(f'Total number of duplicated rows in FAO: {dupe_fao_sum}')

Total number of duplicated rows in FAO: 0


`FAOSTAT Dataset`


In [88]:
# Duplicates in FAOSTAT
dupe_faostat_sum = df_faostat.duplicated().sum()

print(f'Total number of duplicated rows in FAOSTAT: {dupe_faostat_sum}')

Total number of duplicated rows in FAOSTAT: 0


**Explanation & Justification:**

> To remove and check duplicates in both the `FAO` and `FAOSTAT` datasets, the _duplicated_ Pandas method is utilized.

>  By default, the _duplicated_ Pandas method returns a boolean series that denotes duplicate rows. To have a numerical representation of the number of duplicates, we added the _sum_ Pandas method, that denotes its total instead.

#### Standardize Column Names


`FAO Dataset`


In [89]:
df_fao.columns = (
    df_fao.columns
    .str.lower()                                 # Convert to lowercase
    .str.replace(' ', '_')                       # Replace spaces with underscores
    .str.replace(r'[^a-z0-9_]', '', regex=True)  # Remove special characters
)

`FAOSTAT Dataset`


In [90]:
df_faostat.columns = (
    df_faostat.columns
    .str.lower()                
    .str.replace(' ', '_')      
    .str.replace(r'[^a-z0-9_]', '', regex=True)  
)

**Explanation & Justification:**

> To standardize the column names of the `FAO` and `FAOSTAT` datasets, we first converted the column names to lowercase and replaced any spaces with underscores. This ensures consistency in case and format, helping to avoid potential issues related to case sensitivity.

> Additionally, we removed any special characters from the column names, leaving only lowercase letters, numbers, and underscores.

> These standardization steps enhance data consistency, making the datasets easier to work with and reducing the likelihood of errors during data processing.

#### Rename Similar Column Names


In [91]:
# Rename the columns
df_faostat.rename(columns={
    'item'        : 'population_item',
    'item_code'   : 'population_item_code',
    'element'     : 'population_element',
    'element_code': 'population_element_code',
    'unit'        : 'population_unit'
}, inplace=True)

df_faostat.columns

Index(['domain_code', 'domain', 'area_code', 'area', 'population_element_code',
       'population_element', 'population_item_code', 'population_item',
       'year_code', 'year', 'population_unit', 'value', 'flag',
       'flag_description', 'note'],
      dtype='object')

**Explanation & Justification:**

> The `FAO` and `FAOSTAT` datasets share similar column names and merging them could lead to confusion or errors. Thus, we renamed such columns to ensure clarity and avoid potential conflicts during analysis.

> We renamed specific columns in the `FAOSTAT` dataset to make them more descriptive and aligned with the dataset's focus on population data. The columns renamed include:
> - 'item' to 'population_item': This clarifies that the column refers specifically to the population data item (e.g., "Population - Estimated and Projected").
> - 'item_code' to 'population_item_code': By adding "population," it specifies that the code is related to population items.
> - 'element' to 'population_element': This indicates that the column pertains to a specific element within the population data (e.g., "Total Population - Both sexes").
> - 'element_code' to 'population_element_code': This specifies that the code relates to population elements (e.g., "511").
> - 'unit' to 'population_unit': This clearly shows that the column represents the unit of measurement for the population data (e.g., "1000 persons").

### **Data Merging**


In [92]:
df_fao.merge(
    df_faostat, 
    how='inner', 
    on=['area_code', 'area']
).sample(3)

,area_abbreviation,area_code,area,item_code,item,element_code,element,unit,latitude,longitude,y1961,y1962,y1963,y1964,y1965,y1966,y1967,y1968,y1969,y1970,y1971,y1972,y1973,y1974,y1975,y1976,y1977,y1978,y1979,y1980,y1981,y1982,y1983,y1984,y1985,y1986,y1987,y1988,y1989,y1990,y1991,y1992,y1993,y1994,y1995,y1996,y1997,y1998,y1999,y2000,y2001,y2002,y2003,y2004,y2005,y2006,y2007,y2008,y2009,y2010,y2011,y2012,y2013,domain_code,domain,population_element_code,population_element,population_item_code,population_item,year_code,year,population_unit,value,flag,flag_description,note
990,AUT,11,Austria,2577,Palm Oil,5142,Food,1000 tonnes,47.52,14.55,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,1.0,1.0,2.0,3.0,4.0,4.0,3.0,3.0,4.0,4.0,5.0,5.0,5.0,5.0,2.0,4.0,3.0,5.0,6.0,6.0,10.0,5.0,2.0,4.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,7.0,6.0,5.0,8.0,9.0,7.0,7.0,7.0,3,3,OA,Annual population,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2017,2017,1000 persons,8735.453,X,International reliable sources,"UNDESA, Population Division – World Population..."
11006,LAO,120,Lao People's Democratic Republic,2736,"Offals, Edible",5142,Food,1000 tonnes,19.86,102.50,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,4.0,5.0,5.0,5.0,6.0,6.0,6.0,7.0,7.0,7.0,8.0,8.0,7.0,7.0,8.0,8.0,8.0,8.0,9.0,9.0,10.0,10.0,11.0,11.0,11,12,OA,Annual population,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2017,2017,1000 persons,6858.160,X,International reliable sources,"UNDESA, Population Division – World Population..."
7722,GHA,81,Ghana,2582,Maize Germ Oil,5142,Food,1000 tonnes,7.95,-1.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,OA,Annual population,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2017,2017,1000 persons,28833.629,X,International reliable sources,"UNDESA, Population Division – World Population..."


In [93]:
len_merged_df = len(
    df_fao.merge(
        df_faostat, 
        how='inner', 
        on=['area_code', 'area']
    )
)

print(f'Number of rows of FAO dataframe: {len(df_fao)}')
print(f'Number of rows of FAOSTAT dataframe: {len(df_faostat)}')
print(f'Number of rows of merged dataframe: {len_merged_df}')

Number of rows of FAO dataframe: 21477
Number of rows of FAOSTAT dataframe: 231
Number of rows of merged dataframe: 21230


**Explanation & Justification:**

> We merged the two dataframes on both 'area_code' and 'area' not only because these columns are present in both datasets, but also provide a more unique identifier for each row in the merged result.

> We chose an _inner join_ for this merge to avoid introducing null values, since there are combinations of 'area_code' and 'area' that do not exist in the `FAO` and `FAOSTAT` dataframes.

> An inner join allows us to retain only the rows where there is a match in both datasets based on the specified columns. This is crucial for ensuring data consistency and accuracy, as it filters out any records that do not have corresponding matches in both dataframes.

> Unlike other types of joins, such as left or right joins, which would include unmatched records from one side or the other, resulting in null values, the inner join provides a focused dataset with only the relevant records that exist in both sources. This approach not only enhances the accuracy of the match but also improves the uniqueness and reliability of each row in the final merged dataset.

#### Create New Dataframe


In [94]:
merged_df = df_fao.merge(
    df_faostat, 
    how='inner', 
    on=['area_code', 'area'])

merged_df.sample(3)

,area_abbreviation,area_code,area,item_code,item,element_code,element,unit,latitude,longitude,y1961,y1962,y1963,y1964,y1965,y1966,y1967,y1968,y1969,y1970,y1971,y1972,y1973,y1974,y1975,y1976,y1977,y1978,y1979,y1980,y1981,y1982,y1983,y1984,y1985,y1986,y1987,y1988,y1989,y1990,y1991,y1992,y1993,y1994,y1995,y1996,y1997,y1998,y1999,y2000,y2001,y2002,y2003,y2004,y2005,y2006,y2007,y2008,y2009,y2010,y2011,y2012,y2013,domain_code,domain,population_element_code,population_element,population_item_code,population_item,year_code,year,population_unit,value,flag,flag_description,note
2356,BIH,80,Bosnia and Herzegovina,2734,Poultry Meat,5142,Food,1000 tonnes,43.92,17.68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,14.0,19.0,24.0,38.0,33.0,26.0,20.0,15.0,16.0,25.0,23.0,27.0,24.0,22.0,29.0,40.0,44.0,46.0,55.0,62,50,OA,Annual population,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2017,2017,1000 persons,3507.017,X,International reliable sources,"UNDESA, Population Division – World Population..."
3804,CHL,40,Chile,2657,"Beverages, Fermented",5142,Food,1000 tonnes,-35.68,-71.54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,OA,Annual population,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2017,2017,1000 persons,18054.726,X,International reliable sources,"UNDESA, Population Division – World Population..."
3427,CAN,33,Canada,2571,Soyabean Oil,5142,Food,1000 tonnes,56.13,-106.35,67.0,64.0,73.0,83.0,81.0,79.0,79.0,77.0,81.0,101.0,82.0,79.0,95.0,106.0,94.0,108.0,104.0,103.0,111.0,110.0,100.0,98.0,104.0,107.0,110.0,107.0,103.0,90.0,93.0,110.0,109.0,137.0,120.0,89.0,98.0,166.0,158.0,151.0,174.0,169.0,186.0,194.0,206.0,201.0,205.0,182.0,190.0,197.0,190.0,205.0,203.0,197,196,OA,Annual population,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2017,2017,1000 persons,36624.199,X,International reliable sources,"UNDESA, Population Division – World Population..."


**Explanation & Justification:**

> We merged the `FAO` and `FAOSTAT` datasets into a single dataframe, `merged_df,` using an inner join based on the common columns area_code and area.

> This operation ensures that only the rows with matching 'area_code' and 'area' values from both datasets are included in the final merged dataframe.

### **Feature Engineering**


##### Column: Years Existing


In [95]:
year_cols = [col for col in merged_df.columns if col.startswith('y') and col[1:].isdigit()]
merged_df['years_existing'] = merged_df[year_cols].notnull().sum(axis=1)

merged_df[['area_code', 'area', 'years_existing']].sample(3)

,area_code,area,years_existing
220,4,Algeria,53
17267,197,Sierra Leone,53
10267,112,Jordan,53


**Explanation & Justification:**

> We created a new column for _years existing_ to count the number of years for which valid (non-negative) data exists per record in the dataset.


##### Column: Average Production


In [96]:
merged_df['average_production'] = merged_df[year_cols].mean(axis=1)

merged_df[['area_code', 'area', 'years_existing', 'average_production']].sample(3)

,area_code,area,years_existing,average_production
9654,104,Ireland,53,280.094340
8087,89,Guatemala,53,85.094340
5377,167,Czechia,21,5.047619


**Explanation & Justification:**

> We created a new column for _average production_ to calculate the mean production over a specified range of years within the dataset. It summarizes production levels over time per record.


##### Column: Value per Capita


In [97]:
merged_df['population_unit'] = merged_df['population_unit'].str.extract('(\d+)').astype(int)

merged_df['population_unit'].unique()


array([1000])

In [98]:
merged_df[['population_unit', 'value']].sample(3)

,population_unit,value
4280,1000,23626.456
11168,1000,1949.670
13684,1000,17035.938


In [99]:
merged_df['value_per_capita'] = merged_df['value'] / merged_df['population_unit']

merged_df[['value', 'population_unit', 'value_per_capita']].sample(3)

,value,population_unit,value_per_capita
15592,10329.506,1000,10.329506
12450,430.835,1000,0.430835
2671,428.697,1000,0.428697


**Explanation & Justification:**

> We created a new column for _value per capita_ by dividing the 'value' by the 'population_unit.'


#### Column: ISO Alpha-3 Country Code


In [100]:
# Create a function to get the iso_alpha_3 of a country using the PyCountry library

def get_iso_alpha3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except:
        return 'INVALID'
    
merged_df['iso_alpha3'] = merged_df['area'].apply(get_iso_alpha3)
merged_df[['area', 'iso_alpha3']].drop_duplicates().head(3)

,area,iso_alpha3
0,Afghanistan,AFG
83,Albania,ALB
206,Algeria,DZA


In [101]:
# Check if there are areas that has 'INVALID' iso_alpha3
merged_df[merged_df['iso_alpha3'] == 'INVALID'][['area', 'iso_alpha3']].drop_duplicates()

,area,iso_alpha3
2162,Bolivia (Plurinational State of),INVALID
3866,"China, Hong Kong SAR",INVALID
3999,"China, Macao SAR",INVALID
4120,"China, mainland",INVALID
4266,"China, Taiwan Province of",INVALID
9293,Iran (Islamic Republic of),INVALID
15658,Republic of Korea,INVALID
19275,Turkey,INVALID
20631,Venezuela (Bolivarian Republic of),INVALID


In [102]:
# Manually set the ISO alpha-3 of these countries 
manual_iso_mapping = {
    'Bolivia (Plurinational State of)': 'BOL',
    'China, Hong Kong SAR': 'HKG',
    'China, Macao SAR': 'MAC',
    'China, mainland': 'CHN',
    'China, Taiwan Province of': 'TWN',
    'Iran (Islamic Republic of)': 'IRN',
    'Republic of Korea': 'KOR',
    'Turkey': 'TUR',
    'Venezuela (Bolivarian Republic of)': 'VEN',
}

merged_df['iso_alpha3'] = merged_df['area'].map(manual_iso_mapping).fillna(merged_df['iso_alpha3'])

merged_df[merged_df['iso_alpha3'] == 'INVALID'][['area', 'iso_alpha3']].drop_duplicates()

,area,iso_alpha3


In [103]:
merged_df[['area', 'iso_alpha3']].drop_duplicates().sample(3)

,area,iso_alpha3
17446,Slovenia,SVN
11187,Lebanon,LBN
16306,Saint Kitts and Nevis,KNA


**Explanation & Justification:**

> We created a new column for _iso_alpha3_ using `pycountry_convert` library to assign continent for each country.

> In the case of some areas (like Timor-Leste), we manually assigned its continent. This column will be useful for future analysis based on continents.


#### Column: Continent


In [104]:
def country_to_continent(country_alpha3):
    try:
        country_alpha2 = pc.country_alpha3_to_country_alpha2(country_alpha3)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    except:
        return 'Invalid'
    
    return country_continent_name

merged_df['continent'] = merged_df['iso_alpha3'].apply(country_to_continent)
merged_df[['area', 'iso_alpha3', 'continent']].drop_duplicates().sample(3)

,area,iso_alpha3,continent
1942,Benin,BEN,Africa
10825,Kyrgyzstan,KGZ,Asia
17678,South Africa,ZAF,Africa


In [105]:
merged_df[merged_df['continent'] == 'Invalid'][['area', 'iso_alpha3', 'continent']].drop_duplicates()

,area,iso_alpha3,continent
18831,Timor-Leste,TLS,Invalid


In [106]:
# Manually assign continent for Timor-Leste
merged_df.loc[merged_df['iso_alpha3'] == 'TLS', 'continent'] = 'Asia'

merged_df[merged_df['continent'] == 'Invalid'][['area', 'iso_alpha3', 'continent']].drop_duplicates()

,area,iso_alpha3,continent


**Explanation & Justification:**

> We created a new column for _continent_ using `pycountry` library to assign ISO alpha-3 for each country.

> In the case of some areas that were not recognized by the library, we manually assigned their respective ISO alpha-3 based on our research for those countries.

> This column will be essential when creating a choropleth maps.


### **Final Merged Dataframe**


In [107]:
merged_df.sample(5)

,area_abbreviation,area_code,area,item_code,item,element_code,element,unit,latitude,longitude,y1961,y1962,y1963,y1964,y1965,y1966,y1967,y1968,y1969,y1970,y1971,y1972,y1973,y1974,y1975,y1976,y1977,y1978,y1979,y1980,y1981,y1982,y1983,y1984,y1985,y1986,y1987,y1988,y1989,y1990,y1991,y1992,y1993,y1994,y1995,y1996,y1997,y1998,y1999,y2000,y2001,y2002,y2003,y2004,y2005,y2006,y2007,y2008,y2009,y2010,y2011,y2012,y2013,domain_code,domain,population_element_code,population_element,population_item_code,population_item,year_code,year,population_unit,value,flag,flag_description,note,years_existing,average_production,value_per_capita,iso_alpha3,continent
18293,SUR,207,Suriname,2911,Pulses,5521,Feed,1000 tonnes,3.92,-56.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,OA,Annual population,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2017,2017,1000,563.402,X,International reliable sources,"UNDESA, Population Division – World Population...",53,0.037736,0.563402,SUR,South America
18547,CHE,211,Switzerland,2743,Cream,5142,Food,1000 tonnes,46.82,8.23,15.0,16.0,17.0,18.0,20.0,20.0,21.0,22.0,23.0,25.0,26.0,28.0,30.0,30.0,31.0,32.0,35.0,36.0,37.0,38.0,39.0,39.0,40.0,42.0,42.0,43.0,44.0,44.0,44.0,44.0,45.0,45.0,45.0,45.0,45.0,46.0,45.0,45.0,48.0,40.0,37.0,40.0,37.0,35.0,34.0,35.0,35.0,37.0,37.0,38.0,38.0,39,40,OA,Annual population,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2017,2017,1000,8476.005,X,International reliable sources,"UNDESA, Population Division – World Population...",53,35.320755,8.476005,CHE,Europe
15301,PHL,171,Philippines,2582,Maize Germ Oil,5142,Food,1000 tonnes,12.88,121.77,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,6.0,6.0,7.0,6.0,7.0,8.0,8.0,8.0,8.0,8.0,6.0,7.0,6.0,6.0,6.0,6.0,7.0,7.0,6.0,6.0,6.0,7.0,8.0,8.0,8,7,OA,Annual population,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2017,2017,1000,104918.090,X,International reliable sources,"UNDESA, Population Division – World Population...",53,4.962264,104.918090,PHL,Asia
17124,SRB,272,Serbia,2586,"Oilcrops Oil, Other",5142,Food,1000 tonnes,44.02,21.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,4.0,3.0,4.0,3.0,4,4,OA,Annual population,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2017,2017,1000,8790.574,X,International reliable sources,"UNDESA, Population Division – World Population...",8,3.500000,8.790574,SRB,Europe
18881,TLS,176,Timor-Leste,2848,Milk - Excluding Butter,5142,Food,1000 tonnes,-8.87,125.73,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0,2.0,4.0,4.0,4.0,5.0,5.0,6.0,6.0,6.0,7.0,7.0,7.0,8.0,8.0,8.0,8.0,8.0,9.0,9.0,10.0,10.0,10.0,10.0,10.0,9.0,10.0,11.0,27.0,4.0,4.0,3.0,1.0,5.0,12.0,9.0,9.0,9,12,OA,Annual population,511,Total Population - Both sexes,3010,Population - Est. & Proj.,2017,2017,1000,1296.311,X,International reliable sources,"UNDESA, Population Division – World Population...",53,6.283019,1.296311,TLS,Asia


In [108]:
# Check if there are rows with null values (excluding the year columns)
non_year_columns = merged_df.columns.difference(year_cols)
merged_df[merged_df[non_year_columns].isnull().any(axis=1)].head(5)


,area_abbreviation,area_code,area,item_code,item,element_code,element,unit,latitude,longitude,y1961,y1962,y1963,y1964,y1965,y1966,y1967,y1968,y1969,y1970,y1971,y1972,y1973,y1974,y1975,y1976,y1977,y1978,y1979,y1980,y1981,y1982,y1983,y1984,y1985,y1986,y1987,y1988,y1989,y1990,y1991,y1992,y1993,y1994,y1995,y1996,y1997,y1998,y1999,y2000,y2001,y2002,y2003,y2004,y2005,y2006,y2007,y2008,y2009,y2010,y2011,y2012,y2013,domain_code,domain,population_element_code,population_element,population_item_code,population_item,year_code,year,population_unit,value,flag,flag_description,note,years_existing,average_production,value_per_capita,iso_alpha3,continent


**Explanation & Justification:**

> We created a new column for _value per capita_ by dividing the `value` by the `population_unit`.


### **Export to CSV**


In [109]:
merged_df.to_csv('merged_df.csv', index=False)

**Explanation & Justification:**

> The group exported the merged dataframe to a CSV file named merged_df.csv.

> This step ensures that the processed data is saved, allowing the group to continue their work without needing to re-run this part of the code.